# Análisis de Riesgo para el Desarrollo de Pozos Petroleros

## Introducción

### El proyecto tiene como objetivo ayudar a la empresa OilyGiant a elegir la mejor región para abrir 200 nuevos pozos de petróleo. Para esto, se analizan tres regiones diferentes, basándose en datos como la cantidad de petróleo que se podría extraer de cada pozo.

### Primero, se crea un modelo que predice cuánto petróleo podría haber en los nuevos pozos. Luego, se seleccionan los pozos con el mayor potencial y se calcula el beneficio de abrirlos en cada región. Finalmente, se analiza el riesgo de perder dinero usando una técnica llamada bootstrapping, que ayuda a ver diferentes escenarios posibles.

### El objetivo es encontrar cuál de las tres regiones ofrece el mayor beneficio y menor riesgo para la empresa, y así recomendar la mejor opción para abrir los pozos.

## Descargar y preparar los datos

In [1]:
# Importamos las librerías necesarias

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Cargamos los datos de los tres archivos csv en tres dataframes

data_0 = pd.read_csv('geo_data_0.csv')

data_1 = pd.read_csv('geo_data_1.csv')

data_2 = pd.read_csv('geo_data_2.csv')

In [3]:
#Imprimimos los primeros 5 registros de cada dataframe

print(data_0.head())
print()
print(data_1.head())
print()
print(data_2.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [4]:
# Imprimimos la información de cada dataframe y su descripción

print(data_0.info())
print(data_0.describe())
print()

print(data_1.info())
print(data_1.describe())
print()

print(data_2.info())
print(data_2.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.3437

##### Los DataFrame no tienen valores nulos, al analizar la media de la columna 'product' nos podemos dar cuenta que en el df data_1 la media es mucho menor que en los otros 2, en conclusión tiene menos petroleo en promedio que la otras 2 regiones.

In [5]:
print(data_0.drop(columns=['id']).corr())
print()
print(data_1.drop(columns=['id']).corr())
print()
print(data_2.drop(columns=['id']).corr())


               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000

               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000

               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


##### Al parecer la columna que más tiene relación con la columna 'product' es 'f2', teniendo la mayor correlación para las 3 regiones, pero con una casi perfecta en la región 1.

## Entrenar y probar el modelo en cada región

In [6]:
# Eliminamos la columna 'id' de cada dataframe ya que no es necesaria para el análisis

data_0 = data_0.drop(columns=['id'])
data_1 = data_1.drop(columns=['id'])
data_2 = data_2.drop(columns=['id'])

In [7]:
# Dividimos los datos en entrenamiento (75%) y validación (25%)

features_0 = data_0.drop(columns=['product'])
target_0 = data_0['product']

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=12345)

# Verificamos los tamaños de los conjuntos de datos

print(features_train_0.shape)
print(features_valid_0.shape)

(75000, 3)
(25000, 3)


In [8]:
# Creamos el modelo de regresión lineal y lo entrenamos con los datos de entrenamiento

model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)

# Guardamos las predicciones y los valores reales en un dataframe

validation_results_0 = pd.DataFrame({'predictions': predictions_valid_0, 'target': target_valid_0})

print(validation_results_0.head())

# Volumen medio de reservas predicho en la región 0

mean_predicted_volume_0 = validation_results_0['predictions'].mean()
print()
print('Volumen medio de reservas predicho (Región 0):', mean_predicted_volume_0)

# Calculamos el RMSE del modelo

rmse_0 = mean_squared_error(validation_results_0['target'], validation_results_0['predictions']) ** 0.5
print()
print('RMSE (Región 0):', rmse_0)

       predictions      target
71751    95.894952   10.038645
80493    77.572583  114.551489
2655     77.892640  132.603635
53233    90.175134  169.072125
91141    70.510088  122.325180

Volumen medio de reservas predicho (Región 0): 92.59256778438035

RMSE (Región 0): 37.5794217150813


##### El modelo predijo un volumen medio de reservas de 92.59 y tuvo un RMSE de 37.57, mientras más bajo sea el RMSE, mejor será la capacidad del modelo para predecir las reservas de petróleo en nuevos pozos.

In [9]:
# Colocamos los pasos previos en funciones

def split_data(data, target_column, test_size=0.25, random_state=12345):
    features = data.drop(columns=[target_column])
    target = data[target_column]
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=test_size, random_state=random_state)
    return features_train, features_valid, target_train, target_valid

def train_and_predict(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    return predictions_valid

def calculate_rmse(target_valid, predictions_valid):
    return mean_squared_error(target_valid, predictions_valid) ** 0.5

In [10]:
# Aplicamos las funciones a los datos de la región 1

features_train_1, features_valid_1, target_train_1, target_valid_1 = split_data(data_1, 'product')

predictions_valid_1 = train_and_predict(features_train_1, target_train_1, features_valid_1)

validation_results_1 = pd.DataFrame({'predictions': predictions_valid_1, 'target': target_valid_1})

mean_predicted_volume_1 = validation_results_1['predictions'].mean()

rmse_1 = calculate_rmse(validation_results_1['target'], validation_results_1['predictions'])

print(validation_results_1.head())
print()
print('Volumen medio de reservas predicho (Región 1):', mean_predicted_volume_1)
print()
print('RMSE (Región 1):', rmse_1)

       predictions     target
71751    82.663314  80.859783
80493    54.431786  53.906522
2655     29.748760  30.132364
53233    53.552133  53.906522
91141     1.243856   0.000000

Volumen medio de reservas predicho (Región 1): 68.728546895446

RMSE (Región 1): 0.8930992867756169


##### Los resultados muestran que el modelo en la Región 1 tiene un volumen medio de reservas predicho de 68.73 y un RMSE de 0.89, lo que indica que las predicciones están muy cerca de los valores reales.

In [11]:
# Aplicamos las funciones a los datos de la región 2

features_train_2, features_valid_2, target_train_2, target_valid_2 = split_data(data_2, 'product')

predictions_valid_2 = train_and_predict(features_train_2, target_train_2, features_valid_2)

validation_results_2 = pd.DataFrame({'predictions': predictions_valid_2, 'target': target_valid_2})

mean_predicted_volume_2 = validation_results_2['predictions'].mean()

rmse_2 = calculate_rmse(validation_results_2['target'], validation_results_2['predictions'])

print(validation_results_2.head())
print()
print('Volumen medio de reservas predicho (Región 2):', mean_predicted_volume_2)
print()
print('RMSE (Región 2):', rmse_2)

       predictions      target
71751    93.599633   61.212375
80493    75.105159   41.850118
2655     90.066809   57.776581
53233   105.162375  100.053761
91141   115.303310  109.897122

Volumen medio de reservas predicho (Región 2): 94.96504596800492

RMSE (Región 2): 40.02970873393434


##### La región 2 tiene el mayor volumen medio de reservas predicho 94.97, lo que sugiere que es la mejor en términos de producción esperada, sin embargo también tiene el mayor RMSE 40.03, lo que indica que el modelo tiene más variabilidad en sus predicciones.

## Calcular las ganancias

In [12]:
# Calculamos el presupuesto necesario para evitar pérdidas

budget = 100_000_000
wells = 200
price = 4500 
break_even = budget / (wells * price)


print("Unidades necesarias para evitar pérdidas:", break_even)
print("Media de reservas predichas - Región 0:", mean_predicted_volume_0)
print("Media de reservas predichas - Región 1:", mean_predicted_volume_1)
print("Media de reservas predichas - Región 2:", mean_predicted_volume_2)


Unidades necesarias para evitar pérdidas: 111.11111111111111
Media de reservas predichas - Región 0: 92.59256778438035
Media de reservas predichas - Región 1: 68.728546895446
Media de reservas predichas - Región 2: 94.96504596800492


##### 

##### La media de todas las regiones está por debajo de 111.1 unidades, hay un alto riesgo de perdidas. La región con mayor media de reservas tiene posibilidades de ser rentable, pero aún debemos considerar la variabilidad con bootstrapping, es crucial elegir los 200 mejores pozos de los 500 disponibles para mejorar la rentabilidad.

## Calcular la ganancia de un conjunto de pozos

In [13]:
# Definimos una función para calcular la ganancia

def calculate_profit(target, predictions, num_of_wells=200, budget=100_000_000, price=4500):
    predictions_series = pd.Series(predictions, index=target.index)
    
    # Seleccionar los 200 pozos con mayores predicciones
    top_wells = target.loc[predictions_series.nlargest(num_of_wells).index]

    # Calcular la ganancia total
    profit = (top_wells.sum() * price) - budget
    
    return round(profit, 2)

# Calculamos la ganancia para cada región
profit_0 = calculate_profit(target_valid_0, predictions_valid_0)
profit_1 = calculate_profit(target_valid_1, predictions_valid_1)
profit_2 = calculate_profit(target_valid_2, predictions_valid_2)

print(f"Profit Region 0: {profit_0}")
print(f"Profit Region 1: {profit_1}")
print(f"Profit Region 2: {profit_2}")


Profit Region 0: 33208260.43
Profit Region 1: 24150866.97
Profit Region 2: 27103499.64


##### Despues de analizar las ganacias estimadas para cada región se recomienda desarrollar pozos petroleros en la Región 0, ya que prensenta la mayor ganancia esperada en comparación con las otras 2 regiones. Antes de tomar una decisión final, es importante verificar riesgo de pérdidas mediante una prueba bootstrap.

## Calcular riesgos y ganancias para cada región

In [14]:
# Creamos una función para realizar el bootstraping

budget = 100_000_000
wells = 200
price = 4500 
df_region_0 = pd.DataFrame({'pred': predictions_valid_0, 'real': target_valid_0})
df_region_1 = pd.DataFrame({'pred': predictions_valid_1, 'real': target_valid_1})
df_region_2 = pd.DataFrame({'pred': predictions_valid_2, 'real': target_valid_2})

def calcular_profit(unidades):
    
    revenue = unidades * price  
    profit = (revenue - budget) 
    return profit

def bootstrap_profit(df_region, n_samples=1000):
    np.random.seed(12345)  
    values = []

    for _ in range(n_samples):
        sample_df = df_region.sample(n=500, replace=False)
        best_200_sample = sample_df.sort_values(by='pred', ascending=False).head(200)
        unidades_sample = best_200_sample['real'].sum() 
        profit = calcular_profit(unidades_sample)  
        values.append(profit)

    values = np.array(values)  
    profit_promedio = values.mean()  
    percentil_menor = np.percentile(values, 2.5)  
    percentil_mayor = np.percentile(values, 97.5)  
    riesgo_perdida = (values < 0).mean()  

    return profit_promedio, percentil_menor, percentil_mayor, riesgo_perdida

# Calcular bootstrapping para cada región
mean_0, lower_0, upper_0, risk_0 = bootstrap_profit(df_region_0)
print("Región 0,", "profit promedio:", mean_0, "percentil menor:", lower_0, "percentil mayor:", upper_0, "riesgo de perdida:", risk_0)
print()
mean_1, lower_1, upper_1, risk_1 = bootstrap_profit(df_region_1)
print("Región 1,", "profit promedio:", mean_1, "percentil menor:", lower_1, "percentil mayor:", upper_1, "riesgo de perdida:", risk_1)
print()
mean_2, lower_2, upper_2, risk_2 = bootstrap_profit(df_region_2)
print("Región 2,", "profit promedio:", mean_2, "percentil menor:", lower_2, "percentil mayor:", upper_2, "riesgo de perdida:", risk_2)

Región 0, profit promedio: 3807108.9070907477 percentil menor: -1269476.3803180228 percentil mayor: 8796139.67847796 riesgo de perdida: 0.072

Región 1, profit promedio: 4482310.651477294 percentil menor: 708993.8493535467 percentil mayor: 8929852.497000607 riesgo de perdida: 0.014

Región 2, profit promedio: 4027965.8717197105 percentil menor: -1436593.0684827077 percentil mayor: 9630261.544863084 riesgo de perdida: 0.071


##### La región con el menor riesgo de perdidas es la región 1, lo que significa que es la mejor región para invertir ya que tiene el mayor profit promedio de las 3 regiones

##### En comparación, las Región 0 y Región 2 tienen percentiles menores negativos lo que significa que podrían enfrentar pérdidas

##### En el análisis previo, la Región 0 fue la preferida, pero ahora, aplicando la técnica del bootstrapping, la elección cambió, siendo la Región 1 la mejor opción.

## Conclusión

### En este proyecto, analizamos tres posibles regiones para el desarrollo de pozos petroleros. Usamos diferentes métodos, como las predicciones de volumen de reservas y la técnica del bootstrapping, para entender cuál de ellas podría generar más beneficios. Aunque todas las regiones tienen un riesgo de pérdidas, la Región 2 resultó ser la mejor opción, ya que aunque no generará ganancias, sus pérdidas serían menores comparadas con las demás. Por lo tanto, después de evaluar todos los datos, la Región 2 es la que presenta el menor impacto negativo y, por lo tanto, es la más recomendable para desarrollar los pozos.